https://www.kaggle.com/ma7555/schengen-visa-stats

In [1]:
import pandas as pd

In [2]:
df2017 = pd.read_csv('2017-data-for-consulates.csv')
df2018 = pd.read_csv('2018-data-for-consulates.csv')

In [3]:
df2017.head().T
df2017.groupby(df2017['Country where consulate is located'])

,0,1,2,3,4
Schengen State,Austria,Austria,Austria,Austria,Austria
Country where consulate is located,ALBANIA,ALGERIA,ARGENTINA,AUSTRALIA,AZERBAIJAN
Consulate,TIRANA,ALGIERS,BUENOS AIRES,CANBERRA,BAKU
Airport transit visas (ATVs) applied for,NaN,NaN,NaN,NaN,NaN
ATVs issued (including multiple),NaN,NaN,NaN,NaN,NaN
Multiple ATVs issued,NaN,NaN,NaN,NaN,NaN
ATVs not issued,NaN,NaN,NaN,NaN,NaN
Not issued rate for ATVs,NaN,NaN,NaN,NaN,NaN
Uniform visas applied for,57,"2,315",17,"2,305","1,922"
Total uniform visas issued (including MEV) \n,47,"1,658",17,"2,190","1,737"


In [4]:
df2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1884 entries, 0 to 1883
Data columns (total 19 columns):
Schengen State                                                                    1871 non-null object
Country where consulate is located                                                1871 non-null object
Consulate                                                                         1874 non-null object
Airport transit visas (ATVs) applied for                                          358 non-null object
 ATVs issued (including multiple)                                                 302 non-null object
Multiple ATVs issued                                                              211 non-null object
ATVs not issued                                                                   130 non-null object
Not issued rate for ATVs                                                          129 non-null object
Uniform visas applied for                                                 

In [14]:
df2017['Country where consulate is located'].unique()

array(['ALBANIA', 'ALGERIA', 'ARGENTINA', 'AUSTRALIA', 'AZERBAIJAN',
       'BOSNIA AND HERZEGOVINA', 'BRAZIL', 'BULGARIA', 'CANADA', 'CHILE',
       'CHINA', 'COLOMBIA', 'CROATIA', 'CUBA', 'CYPRUS', 'EGYPT',
       'ETHIOPIA', 'FORMER YUGOSLAV REPUBLIC OF MACEDONIA', 'GERMANY',
       'HONG KONG S.A.R.', 'INDIA', 'INDONESIA', 'IRAN', 'IRELAND',
       'ISRAEL', 'JAPAN', 'JORDAN', 'KAZAKHSTAN', 'KENYA', 'KUWAIT',
       'LEBANON', 'MALAYSIA', 'MEXICO', 'MOROCCO', 'NIGERIA', 'PAKISTAN',
       'PERU', 'PHILIPPINES', 'ROMANIA', 'RUSSIAN FEDERATION',
       'SAUDI ARABIA', 'SENEGAL', 'SERBIA', 'SLOVAKIA', 'SLOVENIA',
       'SOUTH AFRICA', 'SOUTH KOREA', 'TAIWAN', 'THAILAND', 'TUNISIA',
       'TURKEY', 'UKRAINE', 'UNITED ARAB EMIRATES', 'UNITED KINGDOM',
       'USA', 'VENEZUELA', 'VIETNAM', 'ANGOLA', 'AUSTRIA', 'BELGIUM',
       'BURKINA FASO', 'BURUNDI', 'CAMEROON',
       'CONGO (DEMOCRATIC REPUBLIC)', "COTE D'IVOIRE", 'FINLAND',
       'FRANCE', 'GREECE', 'HUNGARY', 'ITALY', 'JAMAICA

In [19]:
efi_2018 = pd.read_csv('index2018_data.csv')
efi_2018.sort_values(by = ['2018 Score'], ascending = False, inplace = True)

In [21]:
efi_columns = ['CountryID', 'Country Name', 'Region', 'World Rank', 'Region Rank', \
               '2018 Score', 'Tariff Rate (%)', 'Trade Freedom', 'Population (Millions)', \
              'GDP (Billions, PPP)', 'GDP per Capita (PPP)', 'FDI Inflow (Millions)', \
               'Unemployment (%)', 'Inflation (%)', 'Public Debt (% of GDP)']
efi_2018 = efi_2018[efi_columns]
efi_2018.head(3)

,CountryID,Country Name,Region,World Rank,Region Rank,2018 Score,Tariff Rate (%),Trade Freedom,Population (Millions),"GDP (Billions, PPP)",GDP per Capita (PPP),FDI Inflow (Millions),Unemployment (%),Inflation (%),Public Debt (% of GDP)
70,71,Hong Kong,Asia-Pacific,1.0,1.0,90.2,0.0,90.0,7.4,$429.7,"$58,321.6","108,126.0",3.4,2.6,0.1
146,147,Singapore,Asia-Pacific,2.0,2.0,88.8,0.0,90.0,5.6,$492.6,"$87,855.4","61,596.7",1.8,-0.5,112.0
119,120,New Zealand,Asia-Pacific,3.0,3.0,84.2,1.3,87.4,4.7,$177.0,"$37,294.0","2,291.6",5.2,0.6,29.5


In [9]:
efi_2017 = pd.read_csv('index2017_data.csv')